In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
%matplotlib inline

In [3]:
from ikapati.visualization import visualize as vz
from ikapati.data import io

In [4]:
model_dir_path = "../models/7feacba0-2be0-4761-ba49-854067714f3b/"
model_dir = pathlib.Path(model_dir_path)
data_dir = pathlib.Path("../data/processed")
train_dir = data_dir.joinpath("all")

In [5]:
sub_dirs = list(filter(lambda x: x.is_dir(), [item for item in model_dir.iterdir()]))
sub_dirs

[PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/packaged'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__19_5912'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__19_3845'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__01_5449'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__02_1926'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__01_1232'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__21_5614'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__23_4632'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__04_1552'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__20_2543'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__08_2457')]

In [6]:
for item in sub_dirs[1].iterdir():
    print(item.parts[-1])

3.h5
7.h5
15.h5
model.tflite
final.h5
metadata.json
4.h5
2.h5
5.h5
14.h5
1.h5
6.h5


In [7]:
list(sub_dirs[1].iterdir())

[PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/3.h5'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/7.h5'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/15.h5'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/model.tflite'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/final.h5'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/metadata.json'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/4.h5'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/2.h5'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/5.h5'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/14.h5'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409/1.h5'),
 PosixPath('../model

In [8]:
def is_checkpoint(item):
    return item.stem.isnumeric()

def get_checkpoints(directory):
    return list(filter(is_checkpoint, list(directory.iterdir())))

def get_best_checkpoint(checkpoints):
    checkpoint_numbers = list(map(lambda x: int(x.stem), checkpoints))
    return np.argmax(checkpoint_numbers) if checkpoint_numbers else None

def get_best_checkpoint_in_directory(directory):
    checkpoints = get_checkpoints(directory)
    best_checkpoint = get_best_checkpoint(checkpoints)
    return checkpoints[best_checkpoint] if best_checkpoint else None

def get_best_checkpoint_files(sub_dirs):
    checkpoint_paths = []
    for sub_dir in sub_dirs:
        items = list(sub_dir.iterdir())
        best_checkpoint = get_best_checkpoint_in_directory(sub_dir)
        if best_checkpoint:
            checkpoint_paths.append(best_checkpoint)
    return checkpoint_paths

In [9]:
checkpoint_files = get_best_checkpoint_files(sub_dirs)
model_dir = checkpoint_files[0].parent

In [10]:
model = tf.keras.models.load_model(str(checkpoint_files[0]))

In [16]:
model_metadata_file_path = model_dir.joinpath("metadata.json")
model_metadata = io.read_metadata(str(model_metadata_file_path))
batch_size = model_metadata["arguments"]["batch_size"]
test_data_path = train_dir.joinpath("test.tfrecord")
test_dataset = io.read_dataset(str(test_data_path), batch_size, model_metadata["dataset"]["num_classes"])
steps = model_metadata["dataset"]["file_counts"]["test"] // batch_size

In [17]:
results = model.evaluate(test_dataset, steps=steps)

82/82 [==============================] - 30s 368ms/step - loss: 0.3761 - accuracy: 0.8964


In [ ]:
def evaluate_on_test_data(model_dir_path):
    model_dir = pathlib.Path(model_dir_path)
    sub_dirs = list(filter(lambda x: x.is_dir(), list(model_dir.iterdir())))

    model_metadata_file_path = model_dir.joinpath("metadata.json")
    model_metadata = io.read_metadata(str(model_metadata_file_path))
    
    print_model_info(model_metadata)
    batch_size = model_metadata["batch_size"]
    
    checkpoint_files = get_best_checkpoint_files(sub_dirs)
    
    model = tf.keras.models.load_model(model_path)
    
    test_data_path = train_dir.joinpath("test.tfrecord")
    test_dataset = io.read_dataset(str(test_data_path), batch_size, metadata["num_classes"])
    
    steps = metadata["file_counts"]["test"] // batch_size
    model.evaluate(test_dataset, steps=steps)
    
def print_model_info(model_metadata):
    print(
        "batch_size:", model_metadata["arguments"]["batch_size"], 
        "activation:", model_metadata["arguments"]["epochs"],
        "architecture:", model_metadata["arguments"]["architecture"],
        "dropout:", model_metadata["arguments"]["dropout"],
    )